# 部署和使用ChatYuan
进行中文问答

### SageMaker  Endpoint 部署ChatGLM
  
[ChatGLM](https://github.com/THUDM/ChatGLM-6B): ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。 ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

首先，我们使用Sagemaker部署ChatGLM模型。
#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [ ]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

In [53]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

接下来，我们使用Sagemaker进行模型部署。

In [54]:
from sagemaker.pytorch.model import PyTorchModel

model_name = None
entry_point = 'inference-chatyuan.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

model = PyTorchModel(
    name = model_name,
    model_data = "./model.tar.gz",
    entry_point = entry_point,
    source_dir = './code_chatyuan',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

dummy


然后部署该模型为 Sagemaker endpoint

In [55]:

# deploy as sagemaker endpoint
endpoint_name = "mt-chatyuan-v2-entpoint"
instance_type = 'ml.g4dn.xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

----------!

In [58]:
inputs= [
    {"ask": "写一首关于交通信号灯的诗"},
    {"ask": "陨石为什么总能落在陨石坑里?" },
    {"ask": "为什么爸妈结婚没叫我参加婚礼?"}
]
response = predictor.predict(inputs[0])
print("\n\n问题: ", inputs[0]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(inputs[1])
print("\n\n问题: ", inputs[1]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(inputs[2])
print("\n\n问题: ", inputs[2]["ask"], "\n回答:\n", response["answer"])



问题:  写一首关于交通信号灯的诗 
回答:
 ： 红灯亮起，绿灯亮起。 红灯停，绿红灯停。 绿灯闪，红灯闪。 红绿灯交替，红灯停绿灯又闪。


问题:  陨石为什么总能落在陨石坑里? 
回答:
 陨石坑是陨石坠落的场所，陨石坠落后，在地面上形成陨石坑，陨石坑的表面被陨石撞击后，陨石碎片就会散落在地面，陨石撞击地面后，碎片会沿着陨石撞击的轨迹落下，这些碎片会落在陨石坑中，陨石在撞击地面时，碎片就会落在陨石上，这些碎屑会沉积在陨石坑里，陨石碎屑会慢慢沉积在地表，形成陨石层，陨石层会随着陨石撞击地球，陨石会撞击地球的表面，陨石会在撞击地球时，撞击地球表面的碎片会沉积到陨石坑内，陨石沉积在地面后就会被地球的重力所吸引，陨石就会坠落在陨石层内，这些陨石碎屑就会沉积在地球表面，地球的引力会吸引陨石碎屑，陨石就坠落在地球的地表，地球重力会吸引地球的陨石碎屑沉积在地球上，地球会吸引大量的陨石碎屑在地球表面上，地球就会吸引大量的地球碎屑沉积到地球表面。


问题:  为什么爸妈结婚没叫我参加婚礼? 
回答:
 我和爸妈都结婚好几年了，结婚前我和他们一起住，结婚后我和父母一起住。结婚后，爸妈和我一起住的时候，我和妈妈一起住的，而我和爸爸一起住的是父母和我一起租的房子，所以我爸妈和我一起住在一个小区里，我们住在一个单元里，所以爸妈和我住在一起，我住在他们家，他们住在我家。我爸妈结婚后我们住在一起了，我妈妈和爸爸也结婚了，我们结婚后就一起住在一起。结婚前，爸妈和我们住一起，我爸爸和妈妈也住一起。结婚时，爸妈都结婚了，我父母也结婚了。结婚那天，爸妈在婚礼上，我爸爸妈妈和我一起在婚礼现场，我爸妈和我爸爸一起在，我和我妈妈一起在。婚礼上我们有新人，新人和新人的家人，还有新人的亲朋好友，还有新娘和新郎的亲朋好友。婚礼结束后，我们全家人一起吃晚饭，吃完饭后，我们和新人一起在小区里散步，散步时，我们还和新人和新娘一起合影。婚礼结束，我们一家人一起回家，我回家后，妈妈和妈妈的家人也一起回家了。我回家之后，妈妈和我妈妈的亲戚也一起回来了，我们一起吃饭，吃饭时，妈妈还和妈妈和新娘的亲戚一起合影，妈妈的亲朋好友也一起合影了。婚礼结束了，我们回家了，妈妈就和妈妈亲戚一起回家。我和爸爸和爸爸的亲戚，还有我妈妈的亲人，还有我的亲戚，还有一些我和爸爸妈妈的亲戚。婚礼后，我去了我家，我到了我家后，爸爸和我妈妈就一

也可以通过Endpoint Name进行调用。

In [31]:
import json
import boto3

sagemaker_endpoint_name = "pytorch-inference-2023-04-26-05-44-41-187"

client = boto3.client('runtime.sagemaker')

def query_endpoint(query):
    encoded_json = json.dumps({"ask": query}).encode('utf-8')
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text

In [44]:
query_endpoint("信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：")

'国家名：阿根廷\n人名：梅西'

### ChatYuan 测试
下面，我们还是使用真格资金建立的一套中文语言模型的测试集，来对这个模型进行测试：
https://github.com/zhenbench/z-bench

In [25]:
import pandas as pd
benchdf_common = pd.read_csv("./z-bench/common.samples.csv")

In [47]:
import random
r = random.randint(0, 63)
test_data = benchdf_common.iloc[r]

response = query_endpoint(test_data["Prompt"])
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatYuan:", response)
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `文字生成` 参考回答: nan
Prompt: 我要采访一位科幻作家，创建一个包含 8 个问题的列表
ChatYuan: ，然后在列表中选择一个你认为最合适的问题，然后输入一个文本，然后根据文本给出一个答案。在 Google 搜索框中输入“科幻作家”，然后点击“搜索”按钮，在搜索结果中找到一个答案，然后将答案输入到列表中。
GPT3.5: 当然，以下是一个包含 8 个问题的科幻作家采访问题列表，您可以根据自己的需要进行调整：
您是如何开始写作的，您是从哪些方面汲取灵感的？
您的作品中有哪些主题是您一直感兴趣的？它们是如何反映在您的作品中的？
您写作的过程中最困难的部分是什么？您是如何克服它们的？
您最喜欢的科幻小说或电影是什么？它们对您的写作有何影响？
在您的作品中，您如何创造和发展一个独特的世界和角色？
您对未来科技的发展有何看法？它们是否会影响您的未来作品？
您的读者们如何影响您的写作？您是否与他们交流？
您有什么未来计划？有哪些新的项目或想法？
GPT4: 您是如何对科幻题材产生兴趣的？能否分享一下您的成为科幻作家的起源故事？
您创作科幻作品时，通常从何处寻找灵感？您是否有特定的方法来激发新的故事想法？
您如何看待科学技术在科幻作品中的作用？在您的作品中，科学和技术是如何与叙事相互影响的？
您认为现实世界中的哪些科学发展和技术趋势在未来可能影响科幻作品的主题和设定？
您的作品中是否有涉及到社会、道德或伦理议题？如有，您如何平衡故事情节与这些议题的探讨？
您如何塑造独特、引人入胜的角色？在您的作品中，角色对于推动故事发展的重要性是如何体现的？
您觉得科幻作品在今天的社会中有哪些重要意义？科幻作品能够对现实世界产生哪些积极影响？
您对于新科幻作家有哪些建议？如何在创作过程中保持持续的创新和激情？


#### 进阶类的问题

In [39]:
import pandas as pd
benchdf_emergent = pd.read_csv("./z-bench/emergent.samples.csv")

In [49]:
import random
r = random.randint(0, 209)
test_data = benchdf_emergent.iloc[r]

response = query_endpoint(test_data["Prompt"])
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatYuan:", response)
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `ToM` `心理学` 参考回答: 萨莉会去篮子里找小球。
Prompt: 萨莉把一个小球放到篮子里，然后用布将篮子盖起来后离开。安娜在萨莉走后，将小球从篮中取出放进身边的盒子里。过一会儿，萨莉回来，萨莉会到哪里去找小球呢?
ChatYuan: 安娜会去哪个地方找小球呢?萨莉会去哪找小球?萨丽莎和安娜会去哪里找小球？萨莉和安娜在哪儿找小球，萨莉在哪里找小球。
GPT3.5: 萨莉会去篮子里找小球。因为萨莉不知道安娜已经将小球取出来放到盒子里了，她认为小球仍然在篮子里。
GPT4: 当萨莉回来时，她会在篮子里找小球，因为在她离开之前，她把小球放在了那里。然而，实际上小球已经被安娜移到了盒子里。


#### 垂直类的问题

In [42]:
import pandas as pd
benchdf_specialized = pd.read_csv("./z-bench/specialized.samples.csv")
benchdf_specialized.shape

(39, 5)

In [50]:
import random
r = random.randint(0, 39)
test_data = benchdf_specialized.iloc[r]

response = query_endpoint(test_data["Prompt"])
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatYuan:", response)
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `生物` 参考回答: d
Prompt: 同源结构经常被引用作为自然选择过程的证据。以下所有的结构都是同源结构，除了
(A) 鸟的翅膀和蝙蝠的翅膀
(B) 鲸鱼的鳍和人的手臂
(C) 海豚的胸鳍和海豹的鳍
(D) 昆虫的前腿和狗的前肢
ChatYuan: 
(E) 昆虫的后腿和狗的后肢 
 (F) 鸟类的翅膀和蝙蝠的翅膀 

GPT3.5: 选项 D 昆虫的前腿和狗的前肢不是同源结构。
GPT4: (A) 鸟的翅膀和蝙蝠的翅膀
同源结构是指具有相似结构、起源和发育过程的生物器官，但在不同物种中可能具有不同的功能。这些相似结构表明这些物种可能拥有共同的祖先。鸟的翅膀和蝙蝠的翅膀并非同源结构，而是类似功能的异源结构。虽然它们都是飞行器官，但它们的骨骼结构和起源并不相同。鸟类翅膀的骨骼基于前肢，而蝙蝠的翅膀由手指延长和皮膜连接而成。这种相似性是由于趋同进化，即不同物种独立地演化出相似的适应特征。


In [23]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

阿根廷，梅西。


### 删除Endpoint
如果部署的模型已经不使用，可以删除 Endpoint，避免不必要的开支。

In [52]:
predictor.delete_endpoint()